<a href="https://colab.research.google.com/github/JstnClmnt/NLP-News-Classification/blob/master/English_News_Classification_Using_Bidirectional_Long_Short_Term_Memory_with_Attention_Mechanisms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import pandas as pd
import tensorflow.keras
import os

# Data Processing, Loading

In [0]:
import os
 
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = dirName+"/"+entry
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles        

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
!unzip "/content/drive/Team Drives/NLP Project/English_News_Classification/English_Dataset.zip"

Archive:  /content/drive/Team Drives/NLP Project/English_News_Classification/English_Dataset.zip
replace data/business/001.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

<h4> Convert the data into a pandas dataframe </h4>

In [5]:
import pandas as pd


dirName = 'data';
    
# Get the list of all files in directory tree at given path
listOfFiles = getListOfFiles(dirName)
    

df=pd.DataFrame(columns=["Title","Description","Category"])
# Print the files    
for elem in listOfFiles:
    print("Reading: "+elem)
    file1 = open(elem,"r",encoding="latin-1") 
    sampleNews=file1.read().split("\n")
    newsDesc=""
    for strline in sampleNews[1:len(sampleNews)]:
        newsDesc+=strline
    """
    print("Category: "+elem.split("/")[1])
    print("Title:"+sampleNews[0])
    print("Description:"+newsDesc[0:50])
    """
    dfsample=pd.DataFrame(columns=["Title","Description","Category"],data=[[sampleNews[0],newsDesc,elem.split("/")[1]]])
    df=df.append(dfsample)

Reading: data/sport/145.txt
Reading: data/sport/273.txt
Reading: data/sport/220.txt
Reading: data/sport/191.txt
Reading: data/sport/120.txt
Reading: data/sport/379.txt
Reading: data/sport/112.txt
Reading: data/sport/239.txt
Reading: data/sport/092.txt
Reading: data/sport/179.txt
Reading: data/sport/372.txt
Reading: data/sport/272.txt
Reading: data/sport/136.txt
Reading: data/sport/340.txt
Reading: data/sport/428.txt
Reading: data/sport/414.txt
Reading: data/sport/169.txt
Reading: data/sport/412.txt
Reading: data/sport/366.txt
Reading: data/sport/285.txt
Reading: data/sport/458.txt
Reading: data/sport/284.txt
Reading: data/sport/116.txt
Reading: data/sport/511.txt
Reading: data/sport/119.txt
Reading: data/sport/167.txt
Reading: data/sport/161.txt
Reading: data/sport/134.txt
Reading: data/sport/164.txt
Reading: data/sport/323.txt
Reading: data/sport/270.txt
Reading: data/sport/057.txt
Reading: data/sport/311.txt
Reading: data/sport/390.txt
Reading: data/sport/225.txt
Reading: data/sport/

In [6]:
df=df.reset_index()
df=df.drop(list(df)[0], axis=1)
df["Description"].head()[0]

'Middlesbrough boss Steve McClaren hopes that Stewart Downing did not seriously damage his hamstring during the 2-1 Uefa Cup win over Graz AK.The winger was carried off on a stretcher in the closing stages as Boro booked their place in the last 16. McClaren said: "Stewart has tweaked a hamstring, which is a blow for us. We will assess it in the morning. "But I\'m delighted with the match result. They say in sport you can win ugly - and we certainly did." McClaren was relieved to make it through after a tough encounter at The Riverside."To be fair to Graz, they had a gameplan - they knocked the ball forward, especially in the first half, and we struggled to deal with it. "In the second half we were better but it was frustrating because we got the goal - but one slip and they were back in it." Boro face Sporting Lisbon for a place in the quarter-finals after the Portuguese side saw off Feyenoord on Thursday. And McClaren added: "Sporting are technically very good - Portuguese sides alway

In [7]:
df.head()

,Title,Description,Category
0,Downing injury mars Uefa victory,Middlesbrough boss Steve McClaren hopes that S...,sport
1,Blues slam Blackburn over Savage,Birmingham have confirmed Blackburn made a bid...,sport
2,Curbishley delight for Johansson,Charlton manager Alan Curbishley paid tribute ...,sport
3,Tottenham bid &#163;8m for Forest duo,Nottingham Forest have confirmed they have rec...,sport
4,Barcelona title hopes hit by loss,Barcelona's pursuit of the Spanish title took ...,sport


<h4> Convert the Labels to Integers </h4>

In [8]:
categories=df["Category"].unique().tolist()
cat_dict=dict(zip(categories, range(0,len((categories)))))
print(cat_dict)
for x in categories:
    print(x+": "+str(len(df[df["Category"]==x])))
df["Category"]=df['Category'].map(cat_dict, na_action='ignore')

{'sport': 0, 'business': 1, 'politics': 2, 'tech': 3, 'entertainment': 4}
sport: 511
business: 510
politics: 417
tech: 401
entertainment: 386


<h4> Removing Special Characters </h4>

In [9]:
df['Description'] = df['Description'].str.replace("[^a-zA-Z#]", " ")
df=df.dropna()
df["Description"].head()[0]

'Middlesbrough boss Steve McClaren hopes that Stewart Downing did not seriously damage his hamstring during the     Uefa Cup win over Graz AK The winger was carried off on a stretcher in the closing stages as Boro booked their place in the last     McClaren said   Stewart has tweaked a hamstring  which is a blow for us  We will assess it in the morning   But I m delighted with the match result  They say in sport you can win ugly   and we certainly did   McClaren was relieved to make it through after a tough encounter at The Riverside  To be fair to Graz  they had a gameplan   they knocked the ball forward  especially in the first half  and we struggled to deal with it   In the second half we were better but it was frustrating because we got the goal   but one slip and they were back in it   Boro face Sporting Lisbon for a place in the quarter finals after the Portuguese side saw off Feyenoord on Thursday  And McClaren added   Sporting are technically very good   Portuguese sides always

# Data Cleaning

<h4> 80-20 Train-Test Split </h4>

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df["Description"], df["Category"], test_size=0.20, random_state=42)

train_sum=0
for x in categories:
    print(x+": "+str(len(y_train[y_train==cat_dict[x]])))
    train_sum+=len(y_train[y_train==cat_dict[x]])
print("Total Training Data: "+str(train_sum)+"\n")

test_sum=0
for x in categories:
    print(x+": "+str(len(y_test[y_test==cat_dict[x]])))
    test_sum+=len(y_test[y_test==cat_dict[x]])
print("Total Test Data: "+str(test_sum))

sport: 396
business: 412
politics: 337
tech: 324
entertainment: 311
Total Training Data: 1780

sport: 115
business: 98
politics: 80
tech: 77
entertainment: 75
Total Test Data: 445


### Tokenization

In [11]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

X_train_notcleaned=[word_tokenize(i) for i in X_train]
y_train=y_train.values
X_test_notcleaned=[word_tokenize(i) for i in X_test]
y_test=y_test.values

print(X_train_notcleaned[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['Apple', 'has', 'expanded', 'its', 'iPod', 'family', 'with', 'the', 'release', 'of', 'its', 'next', 'generation', 'of', 'the', 'digital', 'music', 'players', 'Its', 'latest', 'challenges', 'to', 'the', 'growing', 'digital', 'music', 'gadget', 'market', 'include', 'an', 'iPod', 'mini', 'model', 'which', 'can', 'hold', 'GB', 'compared', 'to', 'a', 'previous', 'GB', 'The', 'company', 'which', 'hopes', 'to', 'keep', 'its', 'dominant', 'place', 'in', 'the', 'digital', 'music', 'market', 'also', 'said', 'the', 'gold', 'coloured', 'version', 'of', 'the', 'mini', 'would', 'be', 'dropped', 'A', 'GB', 'version', 'has', 'also', 'been', 'added', 'to', 'the', 'iPod', 'Photo', 'family', 'The', 'latest', 'models', 'have', 'a', 'longer', 'battery', 'life', 'and', 'their', 'prices', 'have', 'been', 'cut', 'by', 'an', 'average', 'of', 'The', 'original', 'iPod', 'took', 'an', 'early', 'lead', '

<h4> Removing of Stop Words and Words with length <3 </h4>

In [0]:
def remove_stop_words(s):
    s_cleaned=[]
    for w in s:
        if w.lower() not in stopwords.words("english") and len(w)>=3:
            s_cleaned.append(w)
 
    return s_cleaned

In [13]:
nltk.download("stopwords")
from nltk.corpus import stopwords
import multiprocessing as mp

X_train,X_test=[],[]

for x in X_train_notcleaned:
  X_train.append(remove_stop_words(x))


for x in X_test_notcleaned:
  X_test.append(remove_stop_words(x))

print(X_train[0])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['Apple', 'expanded', 'iPod', 'family', 'release', 'next', 'generation', 'digital', 'music', 'players', 'latest', 'challenges', 'growing', 'digital', 'music', 'gadget', 'market', 'include', 'iPod', 'mini', 'model', 'hold', 'compared', 'previous', 'company', 'hopes', 'keep', 'dominant', 'place', 'digital', 'music', 'market', 'also', 'said', 'gold', 'coloured', 'version', 'mini', 'would', 'dropped', 'version', 'also', 'added', 'iPod', 'Photo', 'family', 'latest', 'models', 'longer', 'battery', 'life', 'prices', 'cut', 'average', 'original', 'iPod', 'took', 'early', 'lead', 'digital', 'music', 'player', 'market', 'thanks', 'large', 'storage', 'capacity', 'simple', 'design', 'million', 'portable', 'players', 'sold', 'million', 'Apple', 'iPods', 'analysts', 'agree', 'success', 'also', 'integration', 'iTunes', 'online', 'store', 'given', 'company', 'share', 'legal', 'downloa

 <h4> <p> Creating the Vocabulary and word2index </p> </h4>

In [0]:
words = set([])
for s in X_train:
    for w in s:
        words.add(w.lower())
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs

## Preparation of Training and Test Data before fitting into the model

<h4> Importing the GloVe word embeddings</h4>

<p> You can download the word embeddings at https://nlp.stanford.edu/projects/glove/ </p>
<br><b>Note:</b> Choose the 6B and 300D data

In [15]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2019-05-03 20:00:30--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove.6B.zip.1      100%[===================>] 822.24M  13.3MB/s    in 1m 42s  

2019-05-03 20:02:13 (8.07 MB/s) - ‘glove.6B.zip.1’ saved [862182613/862182613]



In [16]:
!unzip "glove.6B.zip"

Archive:  glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
import numpy as np

embeddings_index = dict()
f = open('glove.6B.300d.txt',encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

<h3> Convert all words to lowercase then to integers, then pad the sentences </h3>

In [18]:
train_sentences_X, test_sentences_X = [], []

EMB_DIM=300
num_words=len(word2index)+1
print("Number of Words:"+str(num_words))

for s in X_train:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_X.append(s_int)
    
for s in X_test:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_X.append(s_int)


MAX_LENGTH = len(max(train_sentences_X, key=len))
print("Max Length: "+str(MAX_LENGTH))  # 271

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
#y_train=to_categorical(y_train)
#y_test=to_categorical(y_test)
print(train_sentences_X[0])
print(y_train[0])

Number of Words:25080
Max Length: 2128
[  372 21026 14433 ...     0     0     0]
3


<p> <h3>Creation of Pre-Trained Word Embeddings to be used for the embedding layer </h3> </p>

In [0]:
embedding_matrix=np.zeros((num_words,EMB_DIM))
#print(word2index)
for word,i in word2index.items():
    if i>num_words:
        continue
    embedding_vector=embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector

## Bidirectional LSTMs with Attention Mechanism

From the paper of <i> Attention is All You Need </i>, we have the follow equation:

\begin{align}
Attention(Q,K,V) &  = Softmax(\frac{QK^T}{\sqrt d_k}) V 
\end{align}

In [0]:
from tensorflow.keras.layers import LSTM
SINGLE_ATTENTION_VECTOR = False
TIME_STEPS=MAX_LENGTH

def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = EMB_DIM
    a = Permute((2, 1))(inputs)
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = multiply([inputs, a_probs])
    return output_attention_mul

### Tensorboard callback for visualizations of loss and accuracy

In [0]:
from tensorflow.keras.callbacks import TensorBoard

class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='./Graph', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()

### Bidirectional Long-Short Term Memory without the Attention Mechanisms

In [22]:
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, CuDNNLSTM,LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation,Dropout
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras import regularizers
from tensorflow.keras.initializers import Constant
import numpy as np
from tensorflow.keras.utils import plot_model


model_lstm=Sequential() 
embedding_layer=Embedding(num_words,EMB_DIM,embeddings_initializer=Constant(embedding_matrix),input_length=MAX_LENGTH,trainable=False)
#embedding_layer=Embedding(num_words, 300,mask_zero=True)
inputs=InputLayer(input_shape=(MAX_LENGTH, ))
model_lstm.add(inputs)
model_lstm.add(embedding_layer)
model_lstm.add(Bidirectional(LSTM(128,return_sequences=True)))
model_lstm.add(Dropout(0.3))
model_lstm.add(Bidirectional(LSTM(128)))
model_lstm.add(Dropout(0.3))
model_lstm.add(Dense(len(np.unique(y_train)),activation="softmax"))
model_lstm.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(0.0001),    
              metrics=["accuracy"])
print(model_lstm.summary())
plot_model(model_lstm, to_file='model_lstm.png')


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2128, 300)         7524000   
_________________________________________________________________
bidirectional (Bidirectional (None, 2128, 256)         439296    
_________________________________________________________________
dropout (Dropout)            (None, 2128, 256)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               394240    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)        

In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

es_valacc = EarlyStopping(monitor='val_acc', mode='max',verbose=1,patience=2,min_delta=0.003)
es_loss = EarlyStopping(monitor='loss', mode='min',verbose=1,min_delta=0.003)
history_lstm=model_lstm.fit(train_sentences_X, y_train, validation_data=(test_sentences_X,y_test), batch_size=128,epochs=15,callbacks=[es_loss,es_valacc,TrainValTensorBoard(write_graph=True,log_dir='./Graph_LSTM')])

Train on 1780 samples, validate on 445 samples
Epoch 1/15


In [0]:
#model_lstm.save("LSTM.h5")
scores = model_lstm.evaluate(test_sentences_X,y_test)
print(model_lstm.metrics_names)
print(scores)

In [0]:
# Save the weights
model_lstm.save_weights('lstm_weights.h5')

# Save the model architecture
with open('lstm_architecture.json', 'w') as f:
    f.write(model_lstm.to_json())
    
from tensorflow.model import model_from_json
del model_lstm
# Model reconstruction from JSON file
with open('lstm_architecture.json', 'r') as f:
    model_lstm = model_from_json(f.read())

# Load weights into the new model
model_lstm.load_weights('lstm_weights.h5')
model_lstm.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(0.0001),    
              metrics=["accuracy"])
scores = model_lstm.evaluate(test_sentences_X,y_test)
print(model_lstm.metrics_names)
print(scores)

In [0]:
# Save the weights
model_lstm.save('lstm.h5')
    
del model_lstm
model_lstm=keras.models.load_model("lstm.h5")
scores = model_lstm.evaluate(test_sentences_X,y_test)
print(model_lstm.metrics_names)
print(scores)

In [0]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure()
plt.plot(history_lstm.history['acc'])
plt.plot(history_lstm.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.yticks(np.arange(0,1,step=0.1))
plt.savefig("acc_lstm.png")

plt.figure( )
# Plot training & validation loss values
plt.plot(history_lstm.history['loss'])
plt.plot(history_lstm.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig("loss_lstm.png")

## Bidirectional Long-Short  Term Memory with Attention Mechanisms

In [0]:
from keras.layers import concatenate,add,dot,multiply, Dense,CuDNNLSTM, LSTM, InputLayer, Bidirectional, Embedding, Activation,Dropout
from keras.optimizers import Adam
from keras import regularizers
from keras.initializers import Constant
from keras.utils import plot_model

inputs = Input(shape=(MAX_LENGTH,))
embedding_layer=Embedding(num_words,EMB_DIM,embeddings_initializer=Constant(embedding_matrix),input_length=MAX_LENGTH,trainable=False)
attention_mul = embedding_layer(inputs)
attention_mul = Bidirectional(LSTM(300,return_sequences=True,activation="tanh"))(attention_mul)
attention_mul=Dropout(0.5)(attention_mul)
attention_mul = Bidirectional(LSTM(300,return_sequences=True,activation="tanh"))(attention_mul)
attention_mul = attention_3d_block(attention_mul)
attention_mul=Dropout(0.5)(attention_mul)
attention_mul = Flatten()(attention_mul)
output = Dense(len(np.unique(y_train)),activation="softmax")(attention_mul)
model = Model(inputs=[inputs], outputs=output)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(0.0001),    
              metrics=["accuracy"])
print(model.summary())
plot_model(model, to_file='model_attention.png')

In [0]:
from keras.callbacks import ModelCheckpoint,EarlyStopping

es_valacc = EarlyStopping(monitor='val_acc', mode='max',verbose=1,patience=2,min_delta=0.003)
es_loss = EarlyStopping(monitor='loss', mode='min',verbose=1)
history=model.fit(train_sentences_X, y_train, validation_data=(test_sentences_X,y_test), batch_size=32,epochs=15,callbacks=[es_loss,es_valacc,TrainValTensorBoard(write_graph=True,log_dir='./Graph')])

In [0]:
# Save the weights
model.save_weights('attention_weights.h5')

# Save the model architecture
with open('attention_architecture.json', 'w') as f:
    f.write(model.to_json())
    
from keras.models import model_from_json
del model
# Model reconstruction from JSON file
with open('attention_architecture.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('attention_weights.h5')
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(0.0001),    
              metrics=["accuracy"])
scores = model.evaluate(test_sentences_X,y_test)
print(model.metrics_names)
print(scores)

In [0]:
from keras.models import model_from_json
from keras.optimizers import Adam
# Model reconstruction from JSON file
with open('attention_architecture.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('attention_weights.h5')
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(0.0001),    
              metrics=["accuracy"])
scores = model.evaluate(test_sentences_X,y_test)
print(model.metrics_names)
print(scores)

In [0]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.yticks(np.arange(0,1,step=0.1))
plt.savefig("acc_attention.png")

plt.figure( )
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig("loss_attention.png")

### Working Example

In [0]:
from attention_utils import get_activations, get_data_recurrent
attention_vectors = []

input_text="Oscar-nominated director John Singleton is in intensive care after suffering a stroke. known for movies including Boyz N The Hood and 2 Fast 2 Furious, became unwell on Wednesday, his family said. Following news of the stroke, friends and colleagues sent their best wishes. Rapper Snoop Dogg shared a picture of the pair together on Instagram, writing: Pray 4 my brother. " 
input_text=input_text.replace("[^a-zA-Z#]", " ")
input_text=word_tokenize(input_text)
print(input_text)
input_text_cleaned=[]
for w in input_text:
    if w.lower() not in stopwords.words("english") and len(w)>=3:
        input_text_cleaned.append(w)
print(input_text_cleaned)
input_text_index=[]
for w in input_text_cleaned:
    try:
        input_text_index.append(word2index[w.lower()])
    except KeyError:
        input_text_index.append(word2index['-OOV-'])
print(input_text_index)
input_text_pad=pad_sequences([input_text_index], maxlen=MAX_LENGTH, padding='post')
print(input_text_pad)

print(model.predict(input_text_pad))

In [0]:
attention_vector = np.mean(get_activations(model,input_text_pad,print_shape_only=True,layer_name='attention_vec')[0], axis=2).squeeze()
assert (np.sum(attention_vector) - 1.0) < 1e-5

words_index=sorted(range(len(attention_vector)), key=lambda i: attention_vector[i])[-10:]
print(words_index)
top_words=dict()
for x in words_index:
    if x<len(input_text_cleaned):
        top_words[input_text_cleaned[x]]=attention_vector[x]
print(top_words)
